In [ ]:
!pip install numpy
!pip install pandas

# Nueva sección

In [1]:
from google.colab import drive
import sys
import pandas as pd
import numpy as np

In [2]:
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [21]:
class proyecto:
    def __init__(self):
        self.contador=0
        self.resultado=0
        self.promedios=[]
        self.temp_array=[]
        self.temp_valores=[]
        self.temp1=0
        self.temp2=0
        self.temp3=0
        self.data=[]
        self.users=[]
        self.sources=[]
        self.items_book=dict()
        self.temp_items=dict()
        self.temp_book=dict()
        self.book=dict()
        self.matriz=[]
        self.ratings_g=[]
        self.data_coseno=dict()
        self.user_promedio=dict()
        self.t_cos_ajustado=dict()
        self.t_norm_user=dict()
        self.dato_predic=0
        self.maximo=0
        self.minimo=0
        self.data_slope=dict()
        self.c=dict()
    def promedio_movie_ratings(self):
        self.contador=0
        self.temp1=0
        self.resultado=0
        self.temp_array=[]
        self.temp_valores=[]
        for i in range(len(self.sources)):
          self.temp_array.append(0)
          self.temp_valores.append(0)
        for name,valores in self.book.items():
            for valor in valores.keys():
              self.temp_array[int(np.where(self.sources==valor)[0])]+=valores.get(valor)
              self.temp_valores[int(np.where(self.sources==valor)[0])]+=1
        print(self.temp_array)
        for i in range(len(self.temp_array)):
          self.temp_array[i]=float(self.temp_array[i]/(self.temp_valores[i]))
        self.temp_valores=[]
        self.promedios=self.temp_array
        self.temp_array=[]
    def promedio_users(self):
        self.contador=0
        self.temp1=0
        self.resultado=0
        self.temp_array=[]
        self.temp_valores=[]
        for i in range(len(self.users)):
          self.temp_array.append(0)
          self.temp_valores.append(0)
          for j in range(len(self.sources)):
            if self.book.get(self.users[i]).get(self.sources[j])!=None:
              self.temp_array[i]+=self.book.get(self.users[i]).get(self.sources[j])
              self.temp_valores[i]+=1
        for i in range(len(self.temp_array)):
          self.temp_array[i]=float(self.temp_array[i]/(self.temp_valores[i]))
        self.temp_valores=[]
        self.promedios=self.temp_array
        self.temp_array=[]
    def cos_ajustado_mr(self):
        self.temp1=0;self.temp2=0;self.temp3=0
        self.temp_vector=[]
        for i in range(len(self.sources)-1):
          for m in range(0,i):
            self.temp_vector.append(0.0)
          for j in range(i+1,len(self.sources)):
            for k in range(len(self.users)):
              if self.book.get(self.users[k]).get(self.sources[i])!= None and self.book.get(self.users[k]).get(self.sources[j])!=None:
                self.temp1+=(self.book.get(self.users[k]).get(self.sources[i])-self.promedios[k])*(self.book.get(self.users[k]).get(self.sources[j])-self.promedios[k])
                self.temp2+=(self.book.get(self.users[k]).get(self.sources[i])-self.promedios[k])**2
                self.temp3+=(self.book.get(self.users[k]).get(self.sources[j])-self.promedios[k])**2
          #print(self.temp1);print(self.temp2);print(self.temp3)
            self.temp2=self.temp2**0.5
            self.temp3=self.temp3**0.5
            self.temp_vector.append(self.temp1/(self.temp2*self.temp3))
            self.temp1=0;self.temp2=0;self.temp3=0
          self.matriz.append(self.temp_vector)
          self.temp_vector=[]
    def cos_ajustador_user_mr(self):
      self.temp1=0;self.temp2=0;self.temp3=0
      self.temp_vector=[]
      for i in range(len(self.users)):
        for m in range(0,i):
            self.temp_vector.append(0.0)
        for j in range(i+1,len(self.users)):
          for k in range(len(self.sources)):
            
            if self.book.get(self.users[i]).get(self.sources[k])!= None and self.book.get(self.users[j]).get(self.sources[k])!=None:
              self.temp1+=(self.book.get(self.users[i]).get(self.sources[k])-self.promedios[i])*(self.book.get(self.users[j]).get(self.sources[k])-self.promedios[j])
              self.temp2+=(self.book.get(self.users[i]).get(self.sources[k])-self.promedios[i])**2
              self.temp3+=(self.book.get(self.users[j]).get(self.sources[k])-self.promedios[j])**2
          #print(self.temp1);print(self.temp2);print(self.temp3)
          self.temp2=self.temp2**0.5
          self.temp3=self.temp3**0.5
          self.temp_vector.append(self.temp1/(self.temp2*self.temp3))
          self.temp1=0;self.temp2=0;self.temp3=0
        self.matriz.append(self.temp_vector)
        self.temp_vector=[]
    def normalizacion(self,data):
        self.temp_array=[]
        self.resultado=0
        self.temp1=min(data)
        self.temp2=max(data)
        for i in range(len(data)):
            self.temp_array.append(2*(data[i]-self.temp1)-(self.temp1-self.temp2)/(self.temp1-self.temp2))
    def estructura_movieratings(self,archivo):
        self.data=[]
        self.sources=[]
        self.data=[]
        abrir_archivo=pd.read_csv(archivo,delimiter=',')
        self.sources=abrir_archivo['Unnamed: 0']
        self.users=abrir_archivo.columns[1:]
        #self.users = [str(value for value in range(elements)]
        #print(self.users)
        for i in range(len(self.users)):
          self.temp_array=abrir_archivo[self.users[i]].fillna(0)
          for j in range(len(self.temp_array)):
            if self.temp_array[j]!=0.0:
              self.temp_book[self.sources[j]]=self.temp_array[j]
          self.book[self.users[i]]=self.temp_book
          self.temp_book=dict()
    def estructura_d_grande(self,archivo):
      self.users=[]
      self.sources=[]
      self.ratings_g=[]
      self.users=[]
      self.sources=[]
      self.ratings_g=[]
      #abrir_archivo=pd.read_csv(archivo,delimiter=',')
      for abrir_archivo in pd.read_csv(archivo, delimiter=',',chunksize=1000):
        #print(abrir_archivo.columns())
        self.sources=[]
        self.users=[]
        self.ratings_g=[]
        self.users=np.array(abrir_archivo['User-ID']).astype(str)
        self.sources=np.array(abrir_archivo['ISBN'].astype(str))
        self.ratings_g=np.array(abrir_archivo['Book-Rating'].astype(int))
        #print(self.sources[0])
        for i in range(len(self.users)):
          self.temp_book=dict()
          self.temp_book[self.sources[i]]=self.ratings_g[i]
          #print(self.temp_book)
        #print(self.temp_book)
          if len(self.book)!=0:
            if self.ratings_g[i]!=0:
              if self.book.get(self.users[i])==None:
                self.book[self.users[i]]=self.temp_book
              else:
                self.book[self.users[i]].update(self.temp_book)
          else:
            if self.ratings_g[i]!=0:
              self.book[self.users[i]]=self.temp_book
    def generar_data_inversa(self,archivo):
      self.users=[]
      self.sources=[]
      self.ratings_g=[]
      self.users=[]
      self.sources=[]
      self.ratings_g=[]
      #abrir_archivo=pd.read_csv(archivo,delimiter=',')
      for abrir_archivo in pd.read_csv(archivo, delimiter=',',chunksize=1000):
        #t(abrir_archivo.columns())
        self.sources=[]
        self.users=[]
        self.ratings_g=[]
        self.users=np.array(abrir_archivo['User-ID']).astype(str)
        self.sources=np.array(abrir_archivo['ISBN'].astype(str))
        self.ratings_g=np.array(abrir_archivo['Book-Rating'].astype(int))
        #print(self.sources[0])
        for i in range(len(self.users)):
          self.temp_book=dict()
          self.temp_book[self.users[i]]=self.ratings_g[i]
          #print(self.temp_book)
        #print(self.temp_book)
          if len(self.items_book)!=0:
            if self.ratings_g[i]!=0:
              if self.items_book.get(self.sources[i])==None:
                self.items_book[self.sources[i]]=self.temp_book
              else:
                self.items_book[self.sources[i]].update(self.temp_book)
          else:
            if self.ratings_g[i]!=0:
              self.items_book[self.sources[i]]=self.temp_book
    def promedio_users_grande(self):
        self.contador=0
        self.temp1=0
        self.resultado=0
        self.temp_valores=[]
        self.promedios=[]
        for i in range(len(self.book)):
          self.promedios.append(0)
          self.temp_valores.append(0)
        for usuario,v_datos in self.book.items():
          for key in v_datos.keys():
            self.promedios[self.contador]+=float(v_datos[key])
            self.temp_valores[self.contador]+=1
          self.contador+=1
        cont=0
        for usuario,v_datos in self.book.items():
          self.user_promedio[usuario]=(float(self.promedios[cont]/(self.temp_valores[cont])))
          cont+=1
        self.temp_valores=[]
        self.temp_array=[]
    def cos_ajustador_inversa_grande(self,search_libro):
      self.temp1=0;self.temp2=0;self.temp3=0
      self.temp_vector=[]
      self.users=[]
      self.temp1=0
      self.temp2=0
      self.temp3=0
      self.t_cos_ajustado=dict()
      temp_lib=self.items_book.get(search_libro)
      t_1=set(temp_lib.keys())
      for kx,values in self.items_book.items():
        if kx!=search_libro:
          t_2=set(values.keys())
          temporal=t_1 & t_2
          for kz in temporal:
            print(temp_lib.get(kz),self.user_promedio.get(kz),values.get(kz),self.user_promedio.get(kz))
            self.temp1+=(temp_lib.get(kz)-self.user_promedio.get(kz))*((values.get(kz)-self.user_promedio.get(kz)))
            self.temp2+=(temp_lib.get(kz)-self.user_promedio.get(kz))**2
            self.temp3+=(values.get(kz)-self.user_promedio.get(kz))**2
          #self.temp2=self.temp2**0.5
          #self.temp3=self.temp3**0.5
          if self.temp2!=0 and self.temp3!=0:
            self.t_cos_ajustado[kx]=self.temp1/((self.temp2**0.5)*(self.temp3**0.5))
          self.temp1=0;self.temp2=0;self.temp3=0
    def normalizacion(self,user):
      self.t_norm_user=dict()
      self.t_norm_user=self.book.get(user)
      self.minimo=min(self.t_norm_user.values())
      self.maximo=max(self.t_norm_user.values())
      for key,values in self.t_norm_user.items():
        self.t_norm_user[key]=((2*(self.t_norm_user[key]-self.minimo))-(self.maximo-self.minimo))/(self.maximo-self.minimo)
    def prediccion(self):
      self.dato_predic=0
      t_1=set(self.t_norm_user.keys())
      t_2=set(self.t_cos_ajustado.keys())
      temporal=t_1 &t_2
      print(temporal)
      resultado=0
      division=0
      for key in temporal:
        resultado+=self.t_norm_user.get(key)*self.t_cos_ajustado.get(key)
        division+=abs(self.t_cos_ajustado.get(key))
      self.dato_predic=resultado/division
      print(resultado,division)
    def desnormalizacion(self):
      resultado=((self.dato_predic+1)*(self.maximo-self.minimo))/2+self.minimo
      print(resultado)
    def slope_one(self,search_libro):
      self.temp1=0;self.temp2=0;self.temp3=0
      self.temp_vector=[]
      self.users=[]
      self.temp1=0
      self.c=dict()
      self.data_slope=dict()
      temp_lib=self.items_book.get(search_libro)
      t_1=set(temp_lib.keys())
      for kx,values in self.items_book.items():
        if kx!=search_libro:

          t_2=set(values.keys())
          temporal=t_1 & t_2
          self.c[kx]=len(temporal)
          for kz in temporal:
            self.temp1+=temp_lib.get(kz)-values.get(kz)
          self.data_slope[kx]=self.temp1/2
          self.temp1=0
    def rateuser_slope(self,user):
      self.t_norm_user=dict()
      self.t_norm_user=self.book.get(user)
    def predicion_slope(self):
      t_1=set(self.t_norm_user.keys())
      self.temp1=0
      self.temp2=0
      self.temp3=0
      for key in self.t_norm_user.keys():
        if self.data_slope.get(key)!=None and self.c.get(key)!=None:
          #print(self.t_norm_user.get(key),self.data_slope.get(key),self.c.get(key))
          self.temp1+=(self.t_norm_user.get(key)+(self.data_slope.get(key)))*self.c.get(key)
          self.temp2+=self.c.get(key)
      print(self.temp1/self.temp2)


        




In [ ]:

d_movieratings='/content/drive/My Drive/proyecto_db/BX-Book-Ratingsv2.csv'
b=proyecto()
b.estructura_d_grande(d_movieratings)
b.promedio_users_grande()
b.generar_data_inversa(d_movieratings)
#b.alistar_coseno()
#a.promedio_users_grande()
#for key in b.data_coseno.keys():
#  print(key,b.data_coseno.get(key))

In [ ]:
b.cos_ajustador_inversa_grande("8445071769")

In [ ]:
b.normalizacion("194864")

In [ ]:
b.prediccion()
b.desnormalizacion()

{'8420465666', '8445071777', '8433961845', '8401377013', '8427028282', '8489969280', '8466401555', '8445071408', '8432215007', '8423918688', '807253081X', '8430604855', '1740590732', '8422608286', '0864427530', '8466311815', '8478446400', '8484509796', '0864425856', '1864503157', '1858283485'}
8.56989305047779 20.849465252388942
8.527591468982141


In [ ]:
print(b.dato_predic)

0.4110365875928567


In [18]:
a_movieratings='/content/drive/My Drive/proyecto_db/BX-Book-Ratingsv2.csv'
#abrir_archivo=pd.read_csv(archivo,delimiter=',')
c=proyecto()
c.estructura_d_grande(a_movieratings)
c.promedio_users_grande()
c.generar_data_inversa(a_movieratings)

In [19]:
c.slope_one("8445071769")
c.rateuser_slope("194864")
c.predicion_slope()

7.586206896551724


In [23]:
a_movieratings='/content/drive/My Drive/proyecto_db/ratings_lens.csv'
#abrir_archivo=pd.read_csv(archivo,delimiter=',')
d=proyecto()
d.estructura_d_grande(a_movieratings)
d.promedio_users_grande()
d.generar_data_inversa(a_movieratings)